<a href="https://colab.research.google.com/github/dbrunonascimento/IME/blob/main/RegressaoSoftmaxNewton2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Atividade 2**



- Andréa Longarini - 11112600
- Augusto Cezar Garcia Lozano - 12496820
- Bruno Gonçalves Dias - 12099900
- Diego Bruno dos Santos do Nascimento - 11562547
- Henrique Douglas da Silva - 13047692
- Renato Banzai - 3683321

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# dados de treino
columns = ["atrib1", "atrib2", "atrib3", "atrib4", "class"]
df = pd.read_csv('./dataset_train.txt', delimiter = "\t")
df.columns = columns
df.head()


In [ ]:
# dados de teste 
df_test = pd.read_csv('./dataset_teste.txt', delimiter = "\t")
df_test.columns = columns
df_test.head()  

In [ ]:
# Selecao das caracteristicas dos dados de treino
features = df[["atrib1", "atrib2", "atrib3", "atrib4"]] 
features = features.to_numpy() 
target = df['class'].to_numpy()

target_on_hot = np.array([[int(x==1), int(x==2), int(x==3)] for x in target])


In [ ]:
#  Selecao das caracteristicas dos dados de teste
features_test = df_test[["atrib1", "atrib2", "atrib3", "atrib4"]] 
features_test = features_test.to_numpy() # converts feature set to numpy array
target_test = df_test['class'].to_numpy() # converts target column to numpy array

target_on_hot_test = np.array([[int(x==1), int(x==2), int(x==3)] for x in target_test])


In [ ]:
# Padrão Escalar
def standardScaler(feature_array):
    
    total_cols = feature_array.shape[1] # numero de colunas 
    
    for i in range(total_cols): 
        feature_col = feature_array[:, i]
        
        mean = feature_col.mean() # mean stores mean value for the column
        std = feature_col.std() # std stores standard deviation value for the column 
        feature_array[:, i] = (feature_array[:, i] - mean) / std # standard scaling of each element of the column

In [ ]:

standardScaler(features)
# features  

In [ ]:
standardScaler(features_test)
# features_test

In [ ]:
def softmax(x):
    num = np.exp(x)
    denum = np.sum(np.exp(x))
    final = num/denum 
    return final 

In [ ]:
# creating randomized weights for our linear predictor func
weights = np.random.rand(target_on_hot.shape[1], features.shape[1]+1)
ones = np.ones(len(features))
features_w = np.c_[ones, features]

In [ ]:
weights

array([[0.82712248, 0.98289453, 0.70057852, 0.60733647, 0.86386316],
       [0.79346173, 0.19654172, 0.71972345, 0.9527923 , 0.85177473],
       [0.08114018, 0.84157865, 0.87908636, 0.66997386, 0.06927273]])

In [ ]:
learning_rate= 0.01

def gradient(features_w, y, target_on_hot ): 
    return learning_rate * - (y-target_on_hot).T.dot(features_w)

In [ ]:
def cross_entropy(output, target_on_hot):
    return -np.sum(np.log(output) * (target_on_hot), axis=1) 

In [ ]:
maxInt = 40
cost = []

for it in range(maxInt):
    sW = np.matmul(features_w, weights.T)
  
    s2 = []
    for l  in sW:
      s2.append(softmax(l))

    y = np.array(s2) 

    # temp_y = []
    # for linha  in s:
    #   m = max(linha)
    #   x  =  np.where(linha == m)[0][0]
    #   temp_y.append([int(x==0), int(x==1), int(x==2)])
    #np.array(temp_y) 

    # print('cross_entropy-->', y, target_on_hot,  cross_entropy(y, target_on_hot)) 

    cost.append(np.mean(cross_entropy(y, target_on_hot)))

    print('y:', it,  np.sum(y[:,0]), np.sum(y[:,1]), np.sum(y[:,2]))
  
    weights = weights + gradient(features_w, y, target_on_hot)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(cost)), cost)
plt.show()

Método de Newton 

In [ ]:
x = features_w
weights = np.random.rand(target_on_hot.shape[1], features.shape[1]+1)

Yr1 = np.matmul(x, weights.T)

s2 = []
for l in Yr1:
  s2.append(softmax(l))

Yr = np.array(s2) 

In [ ]:
N = len(features) # quantidade de amostras
k = 1  # indices 
ns = 3 # numero de classes
ne = 4 # numero de entradas

dJ = np.zeros([ne*ns,ne*ns]) 

In [ ]:
for n in range(N): 
  lin = 0
  col = 0 
  for k in range(1, ns): # linhas 
     for m in range(1, ne+1): 
       lin = lin + 1
       col = 0    
       for i in range(1, ns):
         for j in range(1, ne+1): #colunas
            col = col + 1   
            if k == i:           
                delta = 1
            else: 
                delta = 0 
            dJ[lin, col] = dJ[lin, col] - x[n,j] * (delta-y[n,i]) * y[n,k] * x[n,m]


In [ ]:
dJ

In [ ]:
from numpy.linalg import inv, eig

def check_cond(H, ne, ns): 
  
  D, V = eig(H)
  D = np.min(D)

  i = 0
  maxIt = 400

  while D < 0.01:
    i = i + 1 
    H = H + (0.01 * np.eye(ne*ns))
    D, V = eig(H)
    D = np.min(D)
 
  return H 

H = check_cond(dJ,ne,ns)

inv(H)
